<a href="https://colab.research.google.com/github/balnarendrasapa/cybersecurity-ner/blob/master/Fine_tune_for_Cybersecurity_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip MITRE.zip

Archive:  MITRE.zip
   creating: MITRE/
  inflating: MITRE/test.txt          
  inflating: MITRE/train.txt         
  inflating: MITRE/valid.txt         


In [ ]:
!pip -q install accelerate transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00


# Login to the Huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Select the base model and task and batch size

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

# Load the train.txt, test.txt and valid.txt using datasets library.

In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset("text", data_files={"train": "MITRE/train.txt", "test": "MITRE/test.txt", "valid": "MITRE/valid.txt"})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 71001
    })
    test: Dataset({
        features: ['text'],
        num_rows: 20017
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 20342
    })
})

# Dataset Preprocessing

We need to convert the raw dataset to huggingface compatible dataset

In [ ]:
def seperate_sentence_tags(dataset):
    sentences = []
    tags = []
    sentence = []
    tag = []
    for i in dataset:
        if i["text"] != "":
            word, word_tag = i["text"].split("\t")
            sentence += [word]
            tag += [word_tag]
            if word == ".":
                sentences += [sentence]
                tags += [tag]
                sentence = []
                tag = []

    return sentences, tags

In [ ]:
def unique_tags_dict(tags):
    unique_tags = set()
    for i in tags:
        for j in i:
            unique_tags.add(j)

    tags_dict = dict()
    j = 0
    unique_tags = sorted(list(unique_tags))
    for i in unique_tags:
        tags_dict[i] = j
        j += 1

    return unique_tags, tags_dict

In [ ]:
def tag_to_num(tags_dict, tags):
    tags_nums = []
    for i in tags:
        temp = [tags_dict[j] for j in i]
        tags_nums += [temp]
        temp = []
    return tags_nums

In [ ]:
from datasets import Dataset, Features, ClassLabel, Value, Sequence

def dataset_hf(sentences, tags_nums, unique_tags):
    dataset = Dataset.from_dict({
        "id": list(range(len(sentences))),
        "tokens": sentences,
        "ner_tags": tags_nums
        },
        features = Features({
            "id": Value(dtype='string'),
            "ner_tags" : Sequence(ClassLabel(names = list(unique_tags))),
            "tokens": Sequence(Value(dtype = "string"))
        }
    ))
    return dataset

In [ ]:
def dataset_preprocess(dataset):
    sentences, tags = seperate_sentence_tags(dataset)
    unique_tags, tags_dict = unique_tags_dict(tags)
    tags_nums = tag_to_num(tags_dict, tags)
    return dataset_hf(sentences, tags_nums, unique_tags)

In [ ]:
raw_dataset["test"][0]

{'text': 'Why\tO'}

In [ ]:
train_dataset = dataset_preprocess(raw_dataset["train"])
test_dataset = dataset_preprocess(raw_dataset["test"])
validation_dataset = dataset_preprocess(raw_dataset["valid"])

In [ ]:
from datasets import DatasetDict


datasets = DatasetDict({
    'train':train_dataset,
    'test': test_dataset,
    'validation':validation_dataset
})

datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2664
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 717
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 785
    })
})

In [ ]:
datasets.push_to_hub("bnsapa/cybersecurity-ner")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/880 [00:00<?, ?B/s]

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"], num_examples = 5)

,id,tokens,ner_tags
0,557,"[The, sample, has, a, multicomponent, structure, and, can, download, a, payload, or, updates, from, its, C, &, C, server, ,, which, happens, to, be, an, FTP, server, belonging, to, the, free, Russian, web, hosting, service, Ucoz, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,544,"[Gooligan, ,, a, family, of, Android, malware, that, came, to, light, in, November, after, it, compromised, more, than, 1, million, Google, accounts, ,, contained, similar, abilities, to, tamper, with, Google, Play, ratings, .]","[B-Malware, O, O, O, O, B-System, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Organization, O, O, O, O, O, O, O, O, B-System, I-System, O, O]"
2,914,"[List, of, anti-virus, packages, that, are, checked, The, payload, goes, a, long, way, to, protect, itself, and, checks, for, anti-virus, software, installed, on, the, mobile, device, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,728,"[A, snapshot, of, the, code, that, processes, each, VM, opcode, and, the, associate, interpreter, The, presence, of, a, VM, and, virtualized, instruction, blocks, can, be, described, in, simpler, terms, :, Essentially, ,, the, creators, of, FinFisher, interposed, a, layer, of, dynamic, code, translation, (, the, virtual, machine, ), that, makes, analysis, using, regular, tools, practically, impossible, .]","[O, B-Malware, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Malware, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,434,"[No, matter, what, button, is, pressed, ,, the, window, stays, on, top, of, all, other, windows, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [ ]:
print(datasets["train"][1082])

datasets["train"].features

{'id': '1082', 'tokens': ['RuMMS', 'samples', 'and', 'C2', 'servers', 'Figure', '8', 'shows', 'how', 'these', 'samples', ',', 'C2', 'servers', 'and', 'hosting', 'websites', 'are', 'related', 'to', 'each', 'other', ',', 'including', 'when', 'they', 'were', 'compiled', 'or', 'observed', '.'], 'ner_tags': [1, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]}


{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['B-Indicator', 'B-Malware', 'B-Organization', 'B-System', 'B-Vulnerability', 'I-Indicator', 'I-Malware', 'I-Organization', 'I-System', 'I-Vulnerability', 'O'], id=None), length=-1, id=None)}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['The', 'authors', 'are', 'trying', 'to', 'latch', 'onto', 'the', 'popularity', 'of', 'the', 'Super', 'Mario', 'Run', 'game', 'to', 'target', 'eagerly', 'waiting', 'Android', 'users', '.']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'the', 'authors', 'are', 'trying', 'to', 'latch', 'onto', 'the', 'popularity', 'of', 'the', 'super', 'mario', 'run', 'game', 'to', 'target', 'eagerly', 'waiting', 'android', 'users', '.', '[SEP]']


In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(22, 24)

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, None]


In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

24 24


In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 3565, 7986, 2448, 15451, 8059, 1001, 1016, 1516, 2852, 9314, 17364, 9350, 27911, 2015, 2293, 7986, 1998, 20421, 1010, 2021, 2061, 2079, 15451, 8059, 6048, 1012, 102], [101, 1037, 2261, 2420, 2067, 1010, 2057, 2626, 2055, 2019, 11924, 28791, 2099, 23445, 8349, 20540, 2004, 1996, 3565, 7986, 2448, 2208, 2005, 11924, 1012, 102], [101, 2057, 2031, 2179, 2178, 6013, 1997, 15451, 8059, 20540, 2004, 1996, 3565, 7986, 2448, 11924, 10439, 1010, 1998, 2023, 2051, 2009, 2038, 2579, 1996, 2433, 1997, 2852, 9314, 17364, 9350, 1006, 6556, 3229, 23445, 1007, 1012, 102], [101, 6947, 8400, 2626, 2055, 1996, 2852, 9314, 17364, 9350, 2217, 1011, 8209, 2007, 1996, 20421, 2175, 10439, 2067, 1999, 2251, 2355, 1025, 1996, 4489, 2182, 2003, 2008, 2045, 2003, 2053, 2208, 2443, 1999, 1996, 24391, 7427, 1012, 102], [101, 1996, 6048, 2024, 2667, 2000, 25635, 3031, 1996, 6217, 1997, 1996, 3565, 7986, 2448, 2208, 2000, 4539, 17858, 3403, 11924, 5198, 1012, 102]], 'attention_mask': [[1, 1, 1, 1,

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2664 [00:00<?, ? examples/s]

Map:   0%|          | 0/717 [00:00<?, ? examples/s]

Map:   0%|          | 0/785 [00:00<?, ? examples/s]

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['B-Indicator',
 'B-Malware',
 'B-Organization',
 'B-System',
 'B-Vulnerability',
 'I-Indicator',
 'I-Malware',
 'I-Organization',
 'I-System',
 'I-Vulnerability',
 'O']

In [ ]:
from transformers import AutoConfig

In [ ]:
_, tags = seperate_sentence_tags(raw_dataset["train"])
_, tags_dict = unique_tags_dict(tags)

In [ ]:
tags_dict

{'B-Indicator': 0,
 'B-Malware': 1,
 'B-Organization': 2,
 'B-System': 3,
 'B-Vulnerability': 4,
 'I-Indicator': 5,
 'I-Malware': 6,
 'I-Organization': 7,
 'I-System': 8,
 'I-Vulnerability': 9,
 'O': 10}

In [ ]:
ids_dict = {value:key for key, value in tags_dict.items()}

In [ ]:
ids_dict

{0: 'B-Indicator',
 1: 'B-Malware',
 2: 'B-Organization',
 3: 'B-System',
 4: 'B-Vulnerability',
 5: 'I-Indicator',
 6: 'I-Malware',
 7: 'I-Organization',
 8: 'I-System',
 9: 'I-Vulnerability',
 10: 'O'}

In [ ]:
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=len(label_list), label2id=tags_dict, id2label=ids_dict)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, config = config)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"cybersecurity-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

<ipython-input-42-edef154daec7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'System': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
print(labels)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-System', 'I-System', 'I-System', 'O', 'O', 'O', 'O', 'O', 'B-System', 'O', 'O']


In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.249156,0.686989,0.740566,0.712772,0.929324
2,No log,0.202561,0.773254,0.734591,0.753427,0.941984
3,0.211800,0.189506,0.773452,0.793396,0.783297,0.949277
4,0.211800,0.183385,0.778475,0.818868,0.798161,0.951121
5,0.211800,0.205967,0.811339,0.796541,0.803872,0.952186
6,0.050700,0.215343,0.769186,0.822642,0.795016,0.951080
7,0.050700,0.214070,0.786580,0.796226,0.791374,0.950670
8,0.050700,0.219610,0.794201,0.792453,0.793326,0.950793


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1336, training_loss=0.10549285668812826, metrics={'train_runtime': 294.5271, 'train_samples_per_second': 72.36, 'train_steps_per_second': 4.536, 'total_flos': 709028875718064.0, 'train_loss': 0.10549285668812826, 'epoch': 8.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.21960970759391785,
 'eval_precision': 0.7942010715411283,
 'eval_recall': 0.7924528301886793,
 'eval_f1': 0.793325987722336,
 'eval_accuracy': 0.9507928053427296,
 'eval_runtime': 3.0823,
 'eval_samples_per_second': 254.683,
 'eval_steps_per_second': 16.222,
 'epoch': 8.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Indicator': {'precision': 0.8320512820512821,
  'recall': 0.9329180642069956,
  'f1': 0.8796024395753331,
  'number': 2087},
 'Malware': {'precision': 0.8026666666666666,
  'recall': 0.5041876046901173,
  'f1': 0.6193415637860082,
  'number': 597},
 'Organization': {'precision': 0.6354166666666666,
  'recall': 0.3588235294117647,
  'f1': 0.4586466165413534,
  'number': 170},
 'System': {'precision': 0.5527156549520766,
  'recall': 0.6268115942028986,
  'f1': 0.5874363327674025,
  'number': 276},
 'Vulnerability': {'precision': 0.7755102040816326,
  'recall': 0.76,
  'f1': 0.7676767676767676,
  'number': 50},
 'overall_precision': 0.7942010715411283,
 'overall_recall': 0.7924528301886793,
 'overall_f1': 0.793325987722336,
 'overall_accuracy': 0.9507928053427296}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1700352893.f88aac7ea754.772.0:   0%|          | 0.00/9.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1700353196.f88aac7ea754.772.1:   0%|          | 0.00/560 [00:00<?, ?B/s]

'https://huggingface.co/bnsapa/cybersecurity-ner/tree/main/'